# Pauli Hamiltonian ZX Calculus - Example Usage

This notebook demonstrates how to use the refactored `PauliHamiltonianZX` class
for summing Pauli strings and computing time evolution.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pauli_hamiltonian_zx import (
    PauliHamiltonianZX,
    generate_random_pauli_string,
    create_collective_decay_hamiltonian,
    compute_subradiance_decay
)


## Basic Usage: Summing Pauli Strings


In [ ]:
# Create a simple Pauli string Hamiltonian
pauli_strings = [
    (3.0, ["X0", "X1"]),
    (1.0, ["X1", "X2"]),
    (-1.0, ["Z0"]),
    (-1.0, ["Z1"]),
    (-1.0, ["Z2"])
]

# Initialize the Hamiltonian
hamiltonian = PauliHamiltonianZX(pauli_strings)

print(f"Total qubits: {hamiltonian.total_qubits}")
print(f"Number of Pauli terms: {len(pauli_strings)}")


In [ ]:
# Build the ZX graph
graph = hamiltonian.build_graph()
print("Graph built successfully")

# Simplify the graph
simplified_graph = hamiltonian.simplify_graph()
print("Graph simplified")


In [ ]:
# Compute the full matrix representation
matrix = hamiltonian.compute_matrix()
print(f"Matrix shape: {matrix.shape}")
print(f"Matrix is Hermitian: {np.allclose(matrix, matrix.conj().T)}")


In [ ]:
# Compute eigenvalues
eigenvalues = hamiltonian.compute_eigenvalues()
print(f"Eigenvalues: {eigenvalues[:10]}")


## Time Evolution


In [ ]:
# Compute time evolution operator exp(-iHt)
time = 1.0
U = hamiltonian.time_evolution(time)
print(f"Time evolution operator shape: {U.shape}")
print(f"Unitary (U†U = I): {np.allclose(U.conj().T @ U, np.eye(U.shape[0]))}")


In [ ]:
# Evolve an initial state
initial_state = np.zeros(2**hamiltonian.total_qubits)
initial_state[0] = 1.0  # |00...0⟩

evolved_state = hamiltonian.evolve_state(initial_state, time=1.0)
print(f"Initial state norm: {np.linalg.norm(initial_state)}")
print(f"Evolved state norm: {np.linalg.norm(evolved_state)}")


In [ ]:
# Compute expectation value
expectation = hamiltonian.expectation_value(initial_state)
print(f"Expectation value <ψ|H|ψ>: {expectation}")


## Subradiance Collective Decay Application


In [ ]:
# Create Hamiltonian for collective decay (subradiance)
num_atoms = 3  # Small number for demonstration
decay_rate = 1.0
detuning = 0.0

pauli_strings = create_collective_decay_hamiltonian(
    num_atoms, decay_rate, detuning
)

print(f"Created {len(pauli_strings)} Pauli terms for {num_atoms} atoms")
for i, (coeff, gates) in enumerate(pauli_strings[:5]):
    print(f"  Term {i}: {coeff:.3f} * {gates}")


In [ ]:
# Initialize Hamiltonian for subradiance
subradiance_hamiltonian = PauliHamiltonianZX(pauli_strings)

# Compute eigenvalues
eigenvalues = subradiance_hamiltonian.compute_eigenvalues()
print(f"Eigenvalues: {eigenvalues}")


In [ ]:
# Compute decay dynamics
times = np.linspace(0, 5, 100)
times, populations = compute_subradiance_decay(
    num_atoms=num_atoms,
    times=times,
    decay_rate=decay_rate,
    detuning=detuning
)

# Plot the decay
plt.figure(figsize=(10, 6))
plt.plot(times, populations, 'b-', linewidth=2)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Excited State Population', fontsize=12)
plt.title('Subradiance Collective Decay', fontsize=14)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()


## Subradiance Eigenvalues from Equation 4

Compute eigenvalues of H = Σ_{j<k} (F_jk/2) * (X_j X_k + Y_j Y_k)
where indices run over pairs: 01, 02, 03, 12, 13, 23, etc.


In [ ]:
from pauli_hamiltonian_zx import (
    compute_subradiance_eigenvalues,
    compute_F_jk,
    create_collective_decay_hamiltonian
)

# Example 1: Simple case with uniform F_jk
num_atoms = 3
decay_rate = 1.0

# Create Hamiltonian: H = Σ_{j<k} (F_jk/2) * (X_j X_k + Y_j Y_k)
# For pairs: 01, 02, 12 (for 3 atoms)
pauli_strings = create_collective_decay_hamiltonian(
    num_atoms, decay_rate=decay_rate, detuning=0.0, F_matrix=None
)

print(f"Number of atoms: {num_atoms}")
print(f"Number of Pauli terms: {len(pauli_strings)}")
print("\nPauli strings (pairs: 01, 02, 12):")
for i, (coeff, gates) in enumerate(pauli_strings):
    print(f"  {i+1}: {coeff:.3f} * {gates}")


In [ ]:
# Compute eigenvalues
eigenvalues = compute_subradiance_eigenvalues(
    num_atoms=num_atoms,
    F_matrix=None,  # Use uniform decay_rate
    decay_rate=decay_rate,
    detuning=0.0
)

print(f"Eigenvalues (sorted):")
sorted_eigs = np.sort(eigenvalues)
for i, eig in enumerate(sorted_eigs):
    print(f"  {i+1}: {eig:.6f}")

# Plot eigenvalues
plt.figure(figsize=(10, 6))
plt.bar(range(len(sorted_eigs)), sorted_eigs, alpha=0.7, edgecolor='black')
plt.xlabel('Eigenvalue Index', fontsize=12)
plt.ylabel('Eigenvalue', fontsize=12)
plt.title('Subradiance Hamiltonian Eigenvalues', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# Example 2: With custom F_jk matrix
# You can provide your own F_jk values from the paper

num_atoms = 4
# Example F_jk matrix (replace with values from PhysRevA.110.023709.pdf)
F_matrix = np.array([
    [1.0, 0.5, 0.3, 0.2],
    [0.5, 1.0, 0.4, 0.25],
    [0.3, 0.4, 1.0, 0.3],
    [0.2, 0.25, 0.3, 1.0]
])

print(f"F_jk matrix for {num_atoms} atoms:")
print(F_matrix)
print("\nPairs: 01, 02, 03, 12, 13, 23")
for j in range(num_atoms):
    for k in range(j + 1, num_atoms):
        print(f"  F_{j}{k} = {F_matrix[j,k]:.3f}")

# Compute eigenvalues
eigenvalues = compute_subradiance_eigenvalues(
    num_atoms=num_atoms,
    F_matrix=F_matrix,
    decay_rate=1.0,
    detuning=0.0
)

print(f"\nEigenvalues (sorted):")
sorted_eigs = np.sort(eigenvalues)
print(sorted_eigs)


In [ ]:
# Example 3: Compute F_jk from atom positions
# This uses the dipole-dipole interaction formula

num_atoms = 4
decay_rate = 1.0
k0 = 1.0  # Wavevector magnitude

# Example positions (e.g., atoms on a line)
positions = np.array([
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [2.0, 0.0, 0.0],
    [3.0, 0.0, 0.0]
])

# Compute F_jk from positions
F_matrix = compute_F_jk(positions, k0=k0, decay_rate=decay_rate)

print(f"Atom positions:")
for i, pos in enumerate(positions):
    print(f"  Atom {i}: {pos}")

print(f"\nF_jk matrix (computed from positions):")
print(F_matrix)

# Compute eigenvalues
eigenvalues = compute_subradiance_eigenvalues(
    num_atoms=num_atoms,
    positions=positions,
    k0=k0,
    decay_rate=decay_rate,
    detuning=0.0
)

print(f"\nEigenvalues (sorted):")
sorted_eigs = np.sort(eigenvalues)
print(sorted_eigs)


r teh 

In [ ]:
from pauli_hamiltonian_zx import (
    compute_subradiance_eigenvalues,
    compute_subradiance_eigenvalues_numpy,
    compare_eigenvalue_methods
)

# Test with simple case
num_atoms = 3
decay_rate = 1.0

print("Computing eigenvalues using both methods...")
print(f"Number of atoms: {num_atoms}")

# Method 1: ZX Calculus
print("\n1. ZX Calculus method:")
eig_zx = compute_subradiance_eigenvalues(
    num_atoms=num_atoms,
    decay_rate=decay_rate,
    detuning=0.0,
    use_numpy=False
)
print(f"   Eigenvalues: {eig_zx}")

# Method 2: Standard Numpy
print("\n2. Standard Numpy method:")
eig_numpy = compute_subradiance_eigenvalues_numpy(
    num_atoms=num_atoms,
    decay_rate=decay_rate,
    detuning=0.0
)
print(f"   Eigenvalues: {eig_numpy}")

# Compare
print("\n3. Comparison:")
max_diff = np.max(np.abs(np.sort(eig_zx) - np.sort(eig_numpy)))
print(f"   Maximum difference: {max_diff:.2e}")
print(f"   Methods match: {max_diff < 1e-10}")


In [ ]:
# Use the comparison function
eig_zx, eig_numpy, match, max_diff = compare_eigenvalue_methods(
    num_atoms=num_atoms,
    decay_rate=decay_rate,
    detuning=0.0,
    tolerance=1e-10
)

print(f"ZX Calculus eigenvalues: {eig_zx}")
print(f"Numpy eigenvalues: {eig_numpy}")
print(f"\nMethods match: {match}")
print(f"Maximum difference: {max_diff:.2e}")

# Plot comparison
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(eig_zx, eig_numpy, alpha=0.7, s=100)
plt.plot([eig_zx.min(), eig_zx.max()], [eig_zx.min(), eig_zx.max()], 'r--', label='y=x')
plt.xlabel('ZX Calculus Eigenvalues', fontsize=12)
plt.ylabel('Numpy Eigenvalues', fontsize=12)
plt.title('Eigenvalue Comparison', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
differences = np.abs(eig_zx - eig_numpy)
plt.bar(range(len(differences)), differences, alpha=0.7, edgecolor='black')
plt.xlabel('Eigenvalue Index', fontsize=12)
plt.ylabel('Absolute Difference', fontsize=12)
plt.title('Eigenvalue Differences', fontsize=14)
plt.yscale('log')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Test with custom F_jk matrix
num_atoms = 4
F_matrix = np.array([
    [1.0, 0.5, 0.3, 0.2],
    [0.5, 1.0, 0.4, 0.25],
    [0.3, 0.4, 1.0, 0.3],
    [0.2, 0.25, 0.3, 1.0]
])

print(f"Testing with custom F_jk matrix for {num_atoms} atoms")
print(f"F_jk matrix:\n{F_matrix}")

# Compare methods
eig_zx, eig_numpy, match, max_diff = compare_eigenvalue_methods(
    num_atoms=num_atoms,
    F_matrix=F_matrix,
    detuning=0.0,
    tolerance=1e-10
)

print(f"\nZX Calculus eigenvalues: {eig_zx}")
print(f"Numpy eigenvalues: {eig_numpy}")
print(f"\nMethods match: {match}")
print(f"Maximum difference: {max_diff:.2e}")
